# Embeddings con BERT


In este notebook, analizaremos en profundidad las embeddings de palabras producidos por BERT, y veremos cómo comenzar a utilizar BERT produciendo embeddings de frases a partir de un modelo ya entrenado.

Esta notebook está derivada en un 95% derived de blog  ["BERT Word Embeddings Tutorial"](http://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/).

# Introducción




### Qué es BERT?

BERT (Bidirectional Encoder Representations from Transformers), lanzado a finales de 2018, es el modelo que utilizaremos en este tutorial. Es un modelo que aplica mucha de las técnicas que se usan en modelos más complejos más nuevos.

Se puede usar BERT modelos para extraer features a partir de texto, o ajustarlos para una tarea específica (clasificación, reconocimiento de entidades, respuesta a preguntas, etc.) con sus propios datos para generar predicciones.


### Por qué usar embeddings de BERT?

Vamos a utilizar a BERT para genera features, concretamente vectores de embeddings de palabras y frases.

 ¿Qué podemos hacer con estos vectores?

 Estos vectores se utilizan como features de alta calidad para los para entrenar modelos. La manera tradicional en un un modelo BOW es que las palabras se representaban como indices (one-hot encoding) o, de forma más útil, como embeddings únicos para cada palabras como los generados por Word2Vec o Fasttext. BERT ofrece una ventaja sobre modelos como Word2Vec: mientras que cada palabra tiene una representación fija en Word2Vec, independientemente del contexto en el que aparezca, BERT produce representaciones de palabras que se basan en las palabras que las rodean.

Por ejemplo, dadas dos las frases:

"*The man was accused of robbing a bank.*"  y

"*The man went fishing by the bank of the river.*"

Word2Vec produciría los mismos embeddings para las 2 ocurrencias de "*bank*", mientras que con BERT la incrustación de palabras para "*bank*" es diferente en cada oración. Además de capturar diferencias obvias como la polisemia, los embeddings de palabras sensibles al contexto capturan otros tipos de información que resultan en representaciones de características más precisas, lo que a su vez mejora el rendimiento del modelo.


# 1. Cargando un modelo de BERT  Pre-entrenado

Aqui instalamos la versión de BERT en pytorch provista por Hugging Face.

En Google Colab, deberá volver a instalar esta bibliotecas cada ver que se reconecta.

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 2.9 MB/s 
     |████████████████████████████████| 6.6 MB 36.3 MB/s 
     |████████████████████████████████| 120 kB 67.2 MB/s 


Carguemos el modelo BERT pre-entrenado, el tokenizador de BERT, y pytorch para hacer andar todo.
Vamos a utilizar un [modelo ya entrenado para español](https://huggingface.co/mrm8488/bert-spanish-cased-finetuned-ner), sensible a mayúsculas y minúsculas, que es a su vez un modelo destilado de [BETO](https://github.com/dccuchile/beto).


La biblioteca `transformers` de HuggingFace provee varias clases útiles para aplicar BERT a diferentes tareas (token classification, text classification, ...). Aquí usamos un modelo `BertModel` que no fue entrenado para ninguna tarea específica, y es una buena opción para usar BERT solo para extraer embeddings.

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

# Nombre del modelo en  Huggingface
BERT_MODEL = 'mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es'
# Algunos modelos BERT disponibles en español:
#    'mrm8488/bert-spanish-cased-finetuned-ner'
#    'dccuchile/bert-base-spanish-wwm-uncased'
#
# Cargar el mismo tokenizador usado para entrenar el modelo
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL)


# 2. Formato de la Entrada a BERT

El modelo pre-entrenado de BERT espera un formato específico del texto de entrada. Debe contener:

1. Un **token especial, `[SEP]`,** para marcar el fin de una frase, o la separacion entre 2 frases.
2. Un **token expecial, `[CLS]`,** al comienzo (sí, al comienzo) del texto. Este token se usa para entrenar modelos de BERT para clasificación, pero BERT espera este token sin importar para qué es el modelo.
3. Tokens que coinciden con el vocabulario fijo con el que fue entrenadoBERT
4. Los **Token IDs** derivados de los tokens, generado por el tokenizador de  BERT
5. **Mask IDs** (también llamadas "attention masks") que se usan para indicar que parte de la frase de entrada (BERT usa tamaño fijo) son  tokens (representados por un 1) and cuales hay que ignorar  (representeados por un 0).
6. **Segment IDs** (También llamados "token type ids") que son usados para distiguir las 2 frases cuando se usan 2 frases consecutivas *A* y *B*: 0 significa la frase *A* y 1 significa la frase  *B*. Para los modelos de clasificación hay 1 sola frase *A*, para los de completar la frase hay *A* y *B* .  Hay un  token type id por cada token.
7. **Positional Embeddings** que se usan para indicar la posición del token dentro de la frase.

Por suerte la biblioteca  `transformers` se encarga de la mayoría de estos problemas ald usar la función  `tokenizer.encode_plus`.

Para un ejemplo de como usar `tokenizer.encode_plus`, vea la notebook de fine tuning.

## 2.1. Tokens Epeciales
La entrada a BERT pueden ser una o 2 frases consecutivas, y se usa el token especial `[SEP]` para separarlas. El token especial `[CLS]` siempre debe aparecer al comienzo del texto, aun cuando no se use.

Tanto `[SEP]` como `[CLS]` son  *obligatorios siempre*, however, aún si tenemos 1 sola frase y no estamos usando a BERT para clasificar, debido a que así fue entrenado el modelo.

**Ejemplo de 1 sola frase de Entrada**:

`[CLS] The man went to the store. [SEP]`

**Ejemplo de 2 frases de Entrada consecutivas**:

`[CLS] The man went to the store. [SEP] He bought a gallon of milk.`





## 2.2. Tokenization

BERT tiene su propio tokenizador, y necesitamos procesar cualquier frase con el mismo tokenizador usado al entrenar.

**IMPORTANTE:** BERT está entrenado con una ventana de 512 tokens, asi que usará hasta los primeros 512 tokens de la frase y descartará el resto.

In [ ]:
text = "Quiero conseguir los embeddings de esta frase."
marked_text = "[CLS] " + text + " [SEP]"

# Tokenizar la frase con el mismo tokenizador usado para entrenar al modelo BERT.
tokenized_text = tokenizer.tokenize(marked_text)

print (tokenized_text)

['[CLS]', 'quiero', 'conseguir', 'los', 'emb', '##ed', '##ding', '##s', 'de', 'esta', 'frase', '.', '[SEP]']


Como se transforma una palabra en tokens? Veamos como se tokeniza la palabra  "*embeddings*":

In [ ]:
tokenizer.tokenize("embeddings")


['emb', '##ed', '##ding', '##s']

Como se ve la transformación de palabra a tokens no es 1:1; la palabra se dividió en varios n-gramas. Los 2 hashes (##)  que preceden el comienzo de la palabra son la manera de denotar que ese n-grama **no** es prefijo de la palabra que lo contiene. De esta manera, el n-grama '##bed' es diferente del n-grama para "bed"; el primero se usa cuando el n-grama 'bed' occure adentro de una palabra más larga, y el 2do para representar la palabra "bed".

Internamente, BERT usa un tokenizer llamado *WordPiece*. Este tokenizer crea n-gramas hasta un vocabulario de tamaño máximo, y conserva los n-gramas más utiles para representar el texto. BERT típicamente tiene un tamaño máximo de tokens de 30.000. Este vocabulario contiene:

1. palabras completas
2. n-gramas que aparecen o solos o al comienzo de una palabra,
3. n-gramas que no son parte del comienzo de una palabra, que van precedidos de '##',
4. Caracteres sueltos

El resultado es que al ver una encontrar una palabra desconocida, en vez de asignarle a todas las palabras desconocidas el mismo tokene 'OOV' or 'UNK,' la palabra aún puede ser parcialmente representada por los n-gramas que sí apareciero en la colencción de entrada.

Entonces, en vez de asignarle un embedding a cada palabra, BERT el asigna un embedding a cada n-grama de la palabra.

(Para ver más información sobre WordPiece, vea el  [paper original ](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/37842.pdf) y [Neural Machine Translation System](https://arxiv.org/pdf/1609.08144.pdf).)

La llamada a `from_pretrained` descargará el modelo automáticamente.


In [ ]:
# Cargar los pesos del modelo pre-entrenado
model = BertModel.from_pretrained(BERT_MODEL,
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Setear al modelo en modo  "evaluacion", que significa que no va a hacer entrenamiento (backpropagation), solo usar al modelo.
model.eval()


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dens

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

## Ejemplo de algunos tokens (n-gramas):

Estos son algunos ejemplos de tokens en el vocabulario del modelo. Recuerde que los tokens con ## son n-gramas o caracteres individuales.

In [ ]:
list(tokenizer.vocab.keys())[5000:5020]

['terminar',
 '##ht',
 'escal',
 '##izado',
 'lab',
 'ofer',
 'terrorismo',
 'cuentas',
 'accidente',
 'austral',
 '##sen',
 'deberían',
 'volvió',
 'imposible',
 '##idor',
 'miles',
 'visita',
 'brasil',
 'fru',
 '55']

Después de dividir la frase en token, aún tenemos que convertir a la frase en una lista de índices de cada token.



## Cómo codificar una frase en índices de  token:

In [ ]:
# Defino un ejemplo con distintos significados de la palabra "cura" y "caras"
text = "la enfermedad del cura es demasiado cara, no tiene tratamiento conocido, dijo poniendo una cara triste."

# Agregar los tokens especiales.
marked_text = "[CLS] " + text + " [SEP]"

# Tokenizar la frase.
tokenized_text = tokenizer.tokenize(marked_text)

# Mapear los tokens a indices.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Imprimir cada token con su indice.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]             4
la            1,032
enfermedad    4,968
del           1,081
cura          7,933
es            1,028
demasiado     2,668
cara          2,618
,             1,019
no            1,054
tiene         1,394
tratamiento   4,440
conocido      3,638
,             1,019
dijo          1,921
poniendo      8,533
una           1,091
cara          2,618
triste        5,542
.             1,008
[SEP]             5


## 2.3. Attention masks

Las attention mask solo se usan para clasificar, e indican a que tokens hay que prestarle atención: 1 a loss  que NO estan enmascarados (prestales atención), 0 a los que estan enmascarados (ignorarlos).

Dado que vamos a usar BERT para generar embeddings, le vamos a poner 1 a todos los tokens.

In [ ]:
# Marcar cada token como "prestarle atención".
attention_mask = [1] * len(tokenized_text)

# 3. Extrayendo Embeddings



## 3.1. Entendiendo al modelo


BERT es una DNN con 12 capas escondidas (hidden layers). Podemos ver a todas las Las capas escondidas, que está en el código abajo están en la variable `hidden_states`. El contenido de `hidden_states` es un objecto de 4 dimensiones, en el siguiente orden:

1. El número de capa (layer) en la red (13 layers)
2. El número de  batch  (1 sentence, solo usado al entrenar)
3. El indice de palabra o token (21 tokens in our sentence)
4. La dimension del vector de cada token en la capa escondida (768). No confundir con la cantidad máxima de tokens que acepta BERT, que es 512.

Pero ¿por qué hay 13 layers si BERT es un modelo de 12 hidden layers? Respuesta: Hay 13 porque La entrada a BERT son embeddings de tokens, no tokens. La 1ra capa son los embeddings de entrada.

Esto significa que hay 179172 (13 x 18 x 768) valores values en la red para representar la frase.


Veamos un poco que forma tienen las capas escondidas de BERT:

(NOTA al margen: la llamada a  `torch.no_grad` le dice a PyTorch que no construya el grafo para computar derivadas. Dado que aquí no vamos a usar backpropagation (ya está entrenado) esto ahorra memoria.)

In [ ]:
# Convertir indices a vectores de PyTorch
tokens_tensor = torch.tensor([indexed_tokens])
attention_mask_tensor = torch.tensor([attention_mask])

# Hacer pasar al texto por  BERT, y juntar las capas escondidas
with torch.no_grad():

    outputs = model(tokens_tensor, attention_mask_tensor)

    # Evaluating the model will return a different number of objects based on
    # how it's  configured in the `from_pretrained` call earlier. In this case,
    # becase we set `output_hidden_states = True`, the third item will be the
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

In [ ]:
print ("Cantidad de layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Cantidad de batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Cantidad de tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Cantidad de hidden units en cada capa (dimension de embedding):", len(hidden_states[layer_i][batch_i][token_i]))

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 21
Number of hidden units: 768


In [ ]:
# `hidden_states` es una lista de  Python.
print('Tip de los valores de cada capa escondida : ', type(hidden_states))

# Cada capa en la lista es un tensor.
print('Forma del vector para cada capa: ', hidden_states[0].size())

Type of hidden_states:  <class 'tuple'>
Tensor shape for each layer:  torch.Size([1, 21, 768])


In [ ]:
# Concatenar los vectores de todas las capas.
token_embeddings = torch.stack(hidden_states, dim=0)

# El tamaño del embedding va a ser = [13 x 1 x 21 x 768]. Estos numeros salen de:
# 13 = number of capas en la DNN = 1 embedding + 12 encoding layers
# 1 = 1 frase a codificar ("batches", en la jerga de BERT)
# 21 = cantidad de tokens en la frase; si cambia la frase cambia este número
# 768 = dimensiones de cada embedding de 1 token
token_embeddings.size()

torch.Size([13, 1, 21, 768])

Descartemos la dimension  "batches" porque no la necesitamos:.

In [ ]:
# Borra la dimension 1 = "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

torch.Size([13, 21, 768])

Y finalmente, ordenamos las dimensiones como [cantidad de  tokens, cantidad de capas, dimension de embedding en cada capa]:

In [ ]:
# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([21, 13, 768])

## 3.3. Creaando embeddings de palabras y frases

BERT no tiene una "capa que retorna el embedding". Lo que tenemos es las capas escondidas de BERT. Queremos conseguir un vector para cada palabra y para cada frase, pero en cambio para cada token tenemos 13 vectores diferentes (1 por cada), cada uno de largo 768 (la dimension de los emdeddings).

Pero ¿Y que hacemos con esas capas?

Para conseguir embeddings que podamos usar, de alguna manera necesitamos o elegir una capa, o combinar capas.

¿Qué combinación de capas es mejor?

Desafortunadamente la respuesta es "depende", pero hay algunas respuestas que funcionan la mayoría de las veces.


### Generando Vectores para Tokens

Vamos a explorar 2 maneras de crear estos vectores.


#### Método A para generar embeddings de tokens:  Concatenación

 **concatenamos** los vectores re las 4 últimas capas. Como cada capa tiene dimensión 768, por cada palabra nos da un vector de largo  `4 x 768 = 3,072`.

In [ ]:
# Stores the token vectors, with shape [nwords x 3,072]
token_vecs_cat = []

# `token_embeddings` is a [nsentences x nwords x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

print ('Shape is: %d tokens x %d dimensions for each token' % (len(token_vecs_cat), len(token_vecs_cat[0])))

Shape is: 21 tokens x 3072 dimensions for each token


#### Método B para para generar embeddings de tokens: Sumar

Otro método es, en vez de concatenar, sumar las 4 últimas capas, dado que todas las capas generan un vector de dimensión 768 como salida.

In [ ]:
# Stores the token vectors, with shape [nwords x 768]
token_vecs_sum = []

# `token_embeddings` is a [nwords x nlayers x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [nwords x 768] tensor

    # The vector of a word will be the sum of the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)

    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d tokens x %d dimensions for each token' % (len(token_vecs_sum), len(token_vecs_sum[0])))

Shape is: 21 tokens x 768 dimensions for each token


Sin importar como transformemos un token a un embedding (concatenando o sumando), aún necesitamos combinar los embeddings de todos los tokens en 1 solo vector.

### Creando Embeddings para la frase: Promediando


Una manera simple y totalmente empírica de de combinar los vectores de tokens en 1 vector de frase es promediarlos, es promediar los vectores de cada token generados por la anteúltima capa. Esto nos da una representación de la frase de dimensión 768.

In [ ]:
# `hidden_states` is a list of tensors of size [nwords x 768], len(hidden_states) = 13 = layers

# `token_vecs` is a tensor with shape [nwords x 768]
token_vecs = hidden_states[-2][0]
# Calculate the average of all token vectors in the sentence.
sentence_embedding = torch.mean(token_vecs, dim=0)

In [ ]:
print ("Our final sentence embedding vector of shape:", sentence_embedding.size())

Our final sentence embedding vector of shape: torch.Size([768])


## 3.4. Confirmando que los vectores dependen del contexto

Para confirmar que los vectores dependen del contexto,veamos los embeddings generados por  2 ocurrencias de las palabras "cara" y "cura" en la siguiente frase:

""la **cura** de la enfermedad del **cura** es demasiado **cara**, dijo con **cara** triste."

Veamos los indices de las ocurrencias de las palabras "cara"y "cura" en la frase de ejemplo.

## Imprimir los tokens de la frase junto con sus indices:

In [ ]:
for i, token_str in enumerate(tokenized_text):
  print (i, token_str)

0 [CLS]
1 la
2 enfermedad
3 del
4 cura
5 es
6 demasiado
7 cara
8 ,
9 no
10 tiene
11 tratamiento
12 conocido
13 ,
14 dijo
15 poniendo
16 una
17 cara
18 triste
19 .
20 [SEP]


Como se ve, la palabra "cara" en la frase tiene 2 tokens con indices 2 y 17 .

Ahora podemos tratar de imprimir los vectores para compararlos. Para este análisis vamos a usar el metodo en que sumamos las 4 últimas capas (método B) y vamos a imprimir las primeras  10 dimensiones de cada vector.


In [ ]:
print('Primeras 10 dimensiones de los vectores correspondientes a "cara".')
print('')
print('cara (en cdemasiado cara) ', str(token_vecs_sum[7][:10]))
print('cara (en "cara triste")   ', str(token_vecs_sum[17][:10]))

Primeras 10 dimensiones de los vectores correspondientes a "cara".

cara (en cdemasiado cara)  tensor([ 3.5607,  1.4791,  3.2746, -0.8986,  7.8802,  1.3034,  1.6476, -0.6275,
        -2.8693, -3.3488])
cara (en "cara triste")    tensor([-1.7550, -3.7085, -1.6301,  2.0678,  6.7078, -1.3986, -4.5931, -2.3785,
        -0.7294, -0.7507])


## Anexo: Mas detalles

### Diferentes estrategias para combinar capas:


Los BERT autores de BERT exploraron diferentes estrategias para generar los embeddings de tokens:

(Image tomada del blog de [Jay Allamar](http://jalammar.github.io/illustrated-bert/))


![alt text](http://jalammar.github.io/images/bert-feature-extraction-contextualized-embeddings.png)

Si bien la  concatenación de las últimas  4 capas produce los mejores resultados en esta evaluación, en realidad varios de los otros métodos dan resultados cercanos, y cual es mejor depende de cada caso.

Esto ocurre porque las different capas de BERT codifican diferente tipo de información.


### Acerca de los tokens especiales:


Si bien en teoría el token `[CLS]` deberia actuar como una  an "representacióin aggregada de los otros token" para tareas de clasificacíon tasks, esto no es la mejor opción si lo que se quiere es sólo generar embeddings de frase. [De acuerdo a Jacob Devlin, uno de los autores de BERT](https://github.com/google-research/bert/issues/164): "*I'm not sure what these vectors are, since BERT does not generate meaningful sentence vectors. It seems that this is is doing average pooling over the word tokens to get a sentence vector, but we never suggested that this will generate meaningful sentence representations*."

Sin embargo, el [CLS] token sí tiene efecto cuando se toma un modelo de embedding de BERT y se hace fine-tuning para problemas de clasificación. Ahí en la última capa, el valor de este token en la última capa se puede usar para clasificar.



### Palabras fuera del vocabulario de entrenamiento

Como las **palabras fuera del vocabulario de entrenamiento** pueden de todas maneras ser divididas (total o parcialmente) en varios tokens de n-gramas, podemos combinar esas partes para generar un embedding par la palabra.

Las alternativas son:
* Promediar los embeddings (una técnica derivada de otros modelos con tokens de n-gramas como  fasttext), y
* Sumar los embeddings de lon n-gramas.


## Origen de esta notebook
Chris McCormick and Nick Ryan. (2019, May 14). *BERT Word Embeddings Tutorial*. Retrieved from http://www.mccormickml.com
